In [ ]:
# no intention of actually keeping this notebook in the repo, once it's all working good I'll make up a new doc about the category_filter
# I have added it for now to demonstrate that I've got the data filter filter_categories functionality working for numeric inputs
# looking at the deck gl docs: https://deck.gl/docs/api-reference/extensions/data-filter-extension#layer-properties
# it appears that we should be able to use strings for the categories but when I try to use string data the layer simply doesnt work

import geopandas as gpd
import ipywidgets
import pyarrow as pa  # noqa
from shapely.geometry import Point

import lonboard
from lonboard.basemap import CartoBasemap
from lonboard.layer_extension import DataFilterExtension

cat_col = "int_col"
# int_col: works
# float_col: works
# str_col: does NOT work :(
#  as is it will throw an arro3 ValueError: Expected object with __arrow_c_array__ method or implementing buffer protocol.
#  we can avoid the arro3 exception by using pyarrow as the input to get_filter_category when we create the layer:
#    `get_filter_category=pa.array(gdf[cat_col])`
#  but the layer doesn't display and throws a lot of the following WebGL error:
#   GL_INVALID_OPERATION: Vertex shader input type does not match the type of the bound vertex attribute


d = {
    "int_col": [0, 1, 2, 3, 4, 5],
    "float_col": [0.0, 1.5, 0.0, 1.5, 0.0, 1.5],
    "str_col": ["even", "odd", "even", "odd", "even", "odd"],
    "geometry": [
        Point(0, 0),
        Point(1, 1),
        Point(2, 2),
        Point(3, 3),
        Point(4, 4),
        Point(5, 5),
    ],
}
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")

point_layer = lonboard.ScatterplotLayer.from_geopandas(
    gdf,
    get_fill_color=(0, 255, 0),
    radius_min_pixels=10,
    extensions=[
        DataFilterExtension(filter_size=None, category_size=1),
    ],  # no range filter, just a category
    get_filter_category=gdf[cat_col],  # use the cat column for the filter category
)

m = lonboard.Map(layers=[point_layer], basemap_style=CartoBasemap.DarkMatter)

filter_enabled_w = ipywidgets.Checkbox(
    value=True,
    description="Filter Enabled",
)


def on_filter_enabled_change(change):  # noqa
    # when we change the checkbox, toggle filtering on the layer
    point_layer.filter_enabled = filter_enabled_w.value


filter_enabled_w.observe(on_filter_enabled_change, names="value")

cat_selector = ipywidgets.SelectMultiple(  # make a select multiple so we can see interaction on the map
    options=list(gdf[cat_col].unique()),
    value=[list(gdf[cat_col].unique())[0]],
    description="Category",
    disabled=False,
)


def on_cat_selector_change(change) -> None:  # noqa
    # when we change the selector, update the filter on the layer.
    point_layer.filter_categories = cat_selector.value


cat_selector.observe(on_cat_selector_change, names="value")

ipywidgets.VBox([m, filter_enabled_w, cat_selector])

In [ ]:
point_layer2 = lonboard.ScatterplotLayer.from_geopandas(
    gdf,
    get_fill_color=(0, 255, 0),
    radius_min_pixels=10,
    extensions=[
        DataFilterExtension(filter_size=1, category_size=None),
    ],  # no category filter, just a range
    get_filter_value=gdf["int_col"],  # use the int_col for the filter category
)

m2 = lonboard.Map(layers=[point_layer2], basemap_style=CartoBasemap.DarkMatter)
point_layer2.filter_range = [0, 5]
m2

In [ ]:
point_layer2.filter_range = [1, 4]

In [ ]:
point_layer3 = lonboard.ScatterplotLayer.from_geopandas(
    gdf,
    get_fill_color=(0, 255, 0),
    radius_min_pixels=10,
    extensions=[
        DataFilterExtension(filter_size=1, category_size=1),
    ],  # no category filter, just a range
    get_filter_category=gdf[
        "float_col"
    ],  # use the float column for the filter category
    get_filter_value=gdf["int_col"],  # use the int column for the filter category
)

point_layer3.filter_categories = [1.5]
point_layer3.filter_range = [0, 3]
m3 = lonboard.Map(layers=[point_layer3], basemap_style=CartoBasemap.DarkMatter)
m3

In [ ]:
point_layer3.filter_range = [0, 5]